# HW2a: Java Calcuator
- 이름: ???
- 학번: ????????

https://calculatorhistory.net

In [1]:
sealed interface Entry { // entries for arithmetic expressions
    enum BinOp implements Entry { ADD, MULT } // +, *
    enum Delim implements Entry { LPAREN, RPAREN } // (, )
    record Value(int number) implements Entry { }

    /////////////////////////////////////////
    // 아래는 실습 이후에 추가된 메소드 
    /////////////////////////////////////////
    
    static String asString(Entry e) {
        Objects.requireNonNull(e);
        return switch ( e.toString() ) {
                case "ADD" -> "+";
                case "MULT" -> "*";
                case "LPAREN" -> "(";
                case "RPAREN" -> ")";
                default -> ""+((Value)e).number();
        };
    }
}

In [2]:
Entry.BinOp.ADD

ADD

In [3]:
Entry.Delim.LPAREN

LPAREN

In [4]:
new Entry.Value(43)

Value[number=43]

In [5]:
Entry.asString( Entry.BinOp.ADD )

+

In [6]:
Entry.asString( new Entry.Value(43) )

43

In [7]:
import java.util.stream.*;

var v1 = new Entry.Value(43);
var v2 = new Entry.Value(56);
var es = List.of(v1, Entry.BinOp.MULT, Entry.Delim.LPAREN, v2, Entry.BinOp.ADD, v1, Entry.Delim.RPAREN );

System.out.println(es);

es.stream().map(Entry::asString).collect(Collectors.joining(" "))

[Value[number=43], MULT, LPAREN, Value[number=56], ADD, Value[number=43], RPAREN]


43 * ( 56 + 43 )

In [8]:
record Pair<A,B>(A first, B second) { } // because Java standard library does not provide pair data type

interface ICalculator {
    // Adds an entriy (either BinOp, Delim, or Vaule) at the end.
    // Appends legal entry input and return true (returns false for illegal input without adding it).
    boolean appendEntry(Entry e);
    
    // Removes the last added entry; The ◄(x or DEL) button
    // Removes the last input entry and returns true (returns false if not possible)
    boolean removeEntry();
    
    Entry.Value currValue(); // Currently calculated value from the current entries (defaults to null).
    Entry.Value lastValue(); // Last calculated value (defaults to 0).
    
    List<Entry> entries();  // Get current entries.
    List< Pair<List<Entry>, Entry.Value> > history();  // History of calculations.
    
    void clearEntries(); // Clear the current entries; AC button.
    void clearHistory(); // Clear history.
    
    /////////////////////////////////////////
    // 아래는 실습 이후에 추가된 메소드 
    /////////////////////////////////////////
    
    // The = button.
    // Adds the current entries and its calculated current value as a pair into history,
    // when the current entries form a valid expression to calculate the current vaule,
    // and clears the current entries.
    // Returns true for adding valid form of entries and its result value.
    // Returns false, without adding to the history,
    // when the current entries do not form a valid expression.
    boolean addHistory();
}

----
참고자료: https://madplay.github.io/post/call-javascript-function-from-java-using-graalvm

In [9]:
%maven org.graalvm.js:js:19.2.0.1
%maven org.graalvm.js:js-scriptengine:19.2.0.1

import org.graalvm.polyglot.*

In [10]:
try (Context context = Context.create("js")) {

    var value = context.eval("js", "3 + 4 * 10");
    System.out.println( value );
    System.out.println( value.toString().equals("43") );
    System.out.println( value.getClass() ); // 7

} catch (Exception e) {
    System.err.println(e);
}

43
true
class org.graalvm.polyglot.Value


In [11]:
Context context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

var value = context.eval("js", "3 + 4 * 10");
System.out.println( value );
System.out.println( value.toString().equals("43") );
System.out.println( value.getClass() );

int vi = value.asInt();
vi == 43

43
true
class org.graalvm.polyglot.Value


true

In [12]:
Context context = Context.create("js");

try {
    Value value = context.eval("js", "3 + 4 * "); // 사실 여기서 문제가 발생할 수도
    System.out.println( value );
    System.out.println( value.toString().equals("43") );
    System.out.println( value.getClass() );
} catch (Exception e) {
    System.err.println(e);
    System.out.println("이걸 적당히 잘 처리하면 유용하게 활용 가능");
}

org.graalvm.polyglot.PolyglotException: SyntaxError: Unnamed:1:8 Expected an operand but found eof
3 + 4 * 
        ^



이걸 적당히 잘 처리하면 유용하게 활용 가능


----

In [10]:
// ... class Calculator implements ICalculator ...